# 2D SLAM with g2opy

## Setup
First we need to import g2opy to be able to run a 2D SLAM example.
We do this by adding the lib folder inside g2o's source to the python path and
afterwards import g2opy

In [ ]:
import os
import sys
import numpy as np
import math

# Adjust path if needed
lib_directory = os.path.join(os.getcwd(), "../../build/lib")
print(f"lib_directory {lib_directory}")

if not lib_directory in sys.path:
    sys.path.append(lib_directory)

import g2opy

## Download a g2o file to work with the optimizer
We download a data set and save it to a temporary file to operate on this data

In [ ]:
import tempfile
import requests

url = "https://raw.githubusercontent.com/OpenSLAM-org/openslam_g2o/master/data/2d/manhattan3500/manhattanOlson3500.g2o"
tmp_fd, tmp_filename = tempfile.mkstemp(suffix=".g2o", prefix="slam2d_python")
data = requests.get(url)
with os.fdopen(tmp_fd, "wb") as f:
    f.write(data.content)

## Creating an optimizer and solver
Now we are ready to create an optimizer and a solver

In [ ]:
def create_optimizer():
    optimizer = g2opy.SparseOptimizer()
    solver = g2opy.BlockSolverX(g2opy.LinearSolverEigenX())
    solver = g2opy.OptimizationAlgorithmLevenberg(solver)
    optimizer.set_algorithm(solver)
    return optimizer


optimizer = create_optimizer()

## Loading the data
Given the optimizer we now can load data into it. For that purpose, we use the
data which we downloaded above.

In [ ]:
optimizer.clear()
optimizer.load(tmp_filename)
print(f"Number of vertices: {len(optimizer.vertices())}")
print(f"Number of edges {len(optimizer.edges())}")

## Visualization of the initial state
First the generic code to visualize the graph by a plotly figure.

In [ ]:
import plotly.graph_objects as go


def plot_slam2d(optimizer, title, covariances=None):
    def edges_coord(edges, dim):
        for e in edges:
            yield e.vertices()[0].estimate().to_vector()[dim]
            yield e.vertices()[1].estimate().to_vector()[dim]
            yield None

    fig = go.Figure()

    # poses of the vertices
    vertices = optimizer.vertices()
    fig.add_trace(
        go.Scatter(
            x=[v.estimate().to_vector()[0] for v in vertices.values()],
            y=[v.estimate().to_vector()[1] for v in vertices.values()],
            mode="markers",
        )
    )

    # edges
    edges = optimizer.edges()  # get set once to have same order
    fig.add_trace(
        go.Scatter(
            x=list(edges_coord(edges, 0)),
            y=list(edges_coord(edges, 1)),
            mode="lines",
            line=dict(width=0.5),
        )
    )

    if covariances:

        def points_for_cov(cov):
            a = cov[0, 0]
            b = cov[0, 1]
            d = cov[1, 1]

            # get eigen-values
            D = a * d - b * b  # determinant of the matrix
            T = a + d  # Trace of the matrix
            h = math.sqrt(0.25 * (T * T) - D)
            lambda1 = 0.5 * T + h  # solving characteristic polynom using p-q-formula
            lambda2 = 0.5 * T - h

            theta = 0.5 * math.atan2(2.0 * b, a - d)
            rotation_matrix = np.array(
                [[np.cos(theta), np.sin(theta)], [-np.sin(theta), np.cos(theta)]]
            )
            majorAxis = 3.0 * math.sqrt(lambda1)
            minorAxis = 3.0 * math.sqrt(lambda2)
            for alpha in np.linspace(0, math.tau, 32):
                yield np.matmul(
                    rotation_matrix,
                    [
                        majorAxis * math.cos(alpha),
                        minorAxis * math.sin(alpha),
                    ],
                )

        cov_points_x = []
        cov_points_y = []
        for v in optimizer.vertices().values():
            if type(v) != g2opy.VertexSE2 or v.fixed():
                continue
            matrix = covariances.block(v.hessian_index(), v.hessian_index())
            vertex_offset = v.estimate().to_vector()
            for p in points_for_cov(matrix):
                cov_points_x.append(vertex_offset[0] + p[0])
                cov_points_y.append(vertex_offset[1] + p[1])
            cov_points_x.append(None)
            cov_points_y.append(None)

        fig.add_trace(
            go.Scatter(
                x=cov_points_x,
                y=cov_points_y,
                mode="lines",
                line=dict(width=0.5),
            )
        )

    fig.update_yaxes(
        scaleanchor="x",
        scaleratio=1,
    )
    fig.update_layout(go.Layout({"title": title, "showlegend": False}))

    return fig

Below, we can visualize the data

In [ ]:
plot_slam2d(optimizer, "Manhattan - Initial state").show()

## Optimization and visualization
Run the optimization and visualize the result

### The graph

In [ ]:
optimizer.vertices()[0].set_fixed(True)
optimizer.initialize_optimization()
optimizer.optimize(30)

plot_slam2d(optimizer, "Manhattan - Optimized state").show()


### Extract and draw covariance
First, we compute the covariance for all pose vertices.

In [ ]:
cov_vertices = [
    (v.hessian_index(), v.hessian_index())
    for v in optimizer.vertices().values()
    if type(v) == g2opy.VertexSE2 and not v.fixed()
]
covariances, covariances_result = optimizer.compute_marginals(cov_vertices)

Afterwards, we can plot the estimate along with the covariance.

In [ ]:
plot_slam2d(optimizer, "Manhattan - Optimized state with covariances", covariances).show()


## Cleaning up
Delete the temporary file

In [ ]:
os.remove(tmp_filename)